# Exercise 3.6 - TensorBoard: How to visualize with Tensorboard

1. Import all required modules and print version of the most important ones: 

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from  IPython import display
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d
import pandas as pd
import numpy as np
import datetime

import tensorflow as tf
import tensorflow_datasets as tfds

!rm -rf ./logs/ 

# Load the TensorBoard notebook extension
%load_ext tensorboard

TensorFlow version: 2.1.0
TensorFlow datasets version: 2.1.0


2. Dataset import and data preparation

    a) Download custom smaller subset of the original dataset

In [2]:
higgs_path = tf.keras.utils.get_file('HIGGSSmall.csv.gz', 'https://github.com/PacktWorkshops/The-Reinforcement-Learning-Workshop/blob/master/Chapter03/HIGGSSmall.csv.gz?raw=true')

    b) Read CSV dataset into TensorFlow Dataset class and repack it to have tuples (features, labels)

In [3]:
N_TEST = int(1e3)
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(N_TRAIN)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_FEATURES = 28

ds = tf.data.experimental.CsvDataset(higgs_path,[float(),]*(N_FEATURES+1), compression_type="GZIP")

def pack_row(*row):
    label = row[0]
    features = tf.stack(row[1:],1)
    return features, label

packed_ds = ds.batch(N_TRAIN).map(pack_row).unbatch()

    c) Create training, validation and test sets and assign them the BATCH_SIZE parameter

In [4]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
test_ds = packed_ds.skip(N_VALIDATION).take(N_TEST).cache()
train_ds = packed_ds.skip(N_VALIDATION+N_TEST).take(N_TRAIN).cache()

test_ds = test_ds.batch(BATCH_SIZE)
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

3. Model creation and training

    a) Create a decaying learning rate

In [5]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

    b) Define a function to compile a model with
        - Adam optimizer
        - Binary cross entropy as loss function
        
       and fit it on training data using early stopping using validation dataset as well as tensorboard callback

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def compile_and_fit(model, name, max_epochs=3000):
    
    optimizer = tf.keras.optimizers.Adam(lr_schedule)
    
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[
                    tf.keras.losses.BinaryCrossentropy(
                        from_logits=True, name='binary_crossentropy'),
                    'accuracy'])

    model.summary()

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    
    history = model.fit(train_ds,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        epochs=max_epochs,
                        validation_data=validate_ds,
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
                                   tensorboard_callback],
                        verbose=2)
    return history

    c) Create the same large model as before but add regularization items such as L2 regularization and Dropout, then compile it and fit

In [7]:
regularization_model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu', input_shape=(N_FEATURES,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

compile_and_fit(regularization_model, "regularizers/regularization", max_epochs=9000)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               14848     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               2

Epoch 42/9000
20/20 - 1s - loss: 0.7035 - binary_crossentropy: 0.6214 - accuracy: 0.6228 - val_loss: 0.6802 - val_binary_crossentropy: 0.5990 - val_accuracy: 0.6470
Epoch 43/9000
20/20 - 1s - loss: 0.7036 - binary_crossentropy: 0.6232 - accuracy: 0.6165 - val_loss: 0.6854 - val_binary_crossentropy: 0.6058 - val_accuracy: 0.6250
Epoch 44/9000
20/20 - 0s - loss: 0.6989 - binary_crossentropy: 0.6200 - accuracy: 0.6247 - val_loss: 0.6754 - val_binary_crossentropy: 0.5974 - val_accuracy: 0.6210
Epoch 45/9000
20/20 - 1s - loss: 0.6968 - binary_crossentropy: 0.6196 - accuracy: 0.6271 - val_loss: 0.6746 - val_binary_crossentropy: 0.5982 - val_accuracy: 0.6400
Epoch 46/9000
20/20 - 1s - loss: 0.6958 - binary_crossentropy: 0.6201 - accuracy: 0.6266 - val_loss: 0.6730 - val_binary_crossentropy: 0.5981 - val_accuracy: 0.6260
Epoch 47/9000
20/20 - 1s - loss: 0.6941 - binary_crossentropy: 0.6199 - accuracy: 0.6314 - val_loss: 0.6704 - val_binary_crossentropy: 0.5970 - val_accuracy: 0.6230
Epoch 48/9

Epoch 92/9000
20/20 - 0s - loss: 0.6374 - binary_crossentropy: 0.6048 - accuracy: 0.6481 - val_loss: 0.6172 - val_binary_crossentropy: 0.5849 - val_accuracy: 0.6660
Epoch 93/9000
20/20 - 1s - loss: 0.6362 - binary_crossentropy: 0.6041 - accuracy: 0.6484 - val_loss: 0.6190 - val_binary_crossentropy: 0.5871 - val_accuracy: 0.6450
Epoch 94/9000
20/20 - 1s - loss: 0.6336 - binary_crossentropy: 0.6019 - accuracy: 0.6462 - val_loss: 0.6157 - val_binary_crossentropy: 0.5842 - val_accuracy: 0.6670
Epoch 95/9000
20/20 - 0s - loss: 0.6338 - binary_crossentropy: 0.6024 - accuracy: 0.6523 - val_loss: 0.6175 - val_binary_crossentropy: 0.5863 - val_accuracy: 0.6470
Epoch 96/9000
20/20 - 1s - loss: 0.6334 - binary_crossentropy: 0.6023 - accuracy: 0.6534 - val_loss: 0.6166 - val_binary_crossentropy: 0.5858 - val_accuracy: 0.6450
Epoch 97/9000
20/20 - 1s - loss: 0.6356 - binary_crossentropy: 0.6050 - accuracy: 0.6453 - val_loss: 0.6126 - val_binary_crossentropy: 0.5821 - val_accuracy: 0.6740
Epoch 98/9

Epoch 142/9000
20/20 - 0s - loss: 0.6187 - binary_crossentropy: 0.5944 - accuracy: 0.6665 - val_loss: 0.5973 - val_binary_crossentropy: 0.5731 - val_accuracy: 0.6810
Epoch 143/9000
20/20 - 0s - loss: 0.6192 - binary_crossentropy: 0.5950 - accuracy: 0.6570 - val_loss: 0.5981 - val_binary_crossentropy: 0.5739 - val_accuracy: 0.6680
Epoch 144/9000
20/20 - 0s - loss: 0.6157 - binary_crossentropy: 0.5915 - accuracy: 0.6642 - val_loss: 0.6006 - val_binary_crossentropy: 0.5763 - val_accuracy: 0.6590
Epoch 145/9000
20/20 - 1s - loss: 0.6151 - binary_crossentropy: 0.5907 - accuracy: 0.6618 - val_loss: 0.6002 - val_binary_crossentropy: 0.5757 - val_accuracy: 0.6740
Epoch 146/9000
20/20 - 1s - loss: 0.6189 - binary_crossentropy: 0.5944 - accuracy: 0.6556 - val_loss: 0.6016 - val_binary_crossentropy: 0.5772 - val_accuracy: 0.6740
Epoch 147/9000
20/20 - 1s - loss: 0.6185 - binary_crossentropy: 0.5940 - accuracy: 0.6628 - val_loss: 0.5964 - val_binary_crossentropy: 0.5719 - val_accuracy: 0.6670
Epoc

Epoch 192/9000
20/20 - 0s - loss: 0.6128 - binary_crossentropy: 0.5872 - accuracy: 0.6630 - val_loss: 0.5953 - val_binary_crossentropy: 0.5696 - val_accuracy: 0.6810
Epoch 193/9000
20/20 - 0s - loss: 0.6151 - binary_crossentropy: 0.5894 - accuracy: 0.6752 - val_loss: 0.6018 - val_binary_crossentropy: 0.5761 - val_accuracy: 0.6830
Epoch 194/9000
20/20 - 0s - loss: 0.6128 - binary_crossentropy: 0.5871 - accuracy: 0.6669 - val_loss: 0.6005 - val_binary_crossentropy: 0.5748 - val_accuracy: 0.6550
Epoch 195/9000
20/20 - 0s - loss: 0.6152 - binary_crossentropy: 0.5894 - accuracy: 0.6629 - val_loss: 0.5943 - val_binary_crossentropy: 0.5685 - val_accuracy: 0.6920
Epoch 196/9000
20/20 - 1s - loss: 0.6149 - binary_crossentropy: 0.5890 - accuracy: 0.6670 - val_loss: 0.5988 - val_binary_crossentropy: 0.5729 - val_accuracy: 0.6640
Epoch 197/9000
20/20 - 1s - loss: 0.6111 - binary_crossentropy: 0.5852 - accuracy: 0.6752 - val_loss: 0.5979 - val_binary_crossentropy: 0.5720 - val_accuracy: 0.6780
Epoc

...

Epoch 892/9000
20/20 - 1s - loss: 0.5923 - binary_crossentropy: 0.5574 - accuracy: 0.6904 - val_loss: 0.5747 - val_binary_crossentropy: 0.5397 - val_accuracy: 0.7000
Epoch 893/9000
20/20 - 1s - loss: 0.5946 - binary_crossentropy: 0.5597 - accuracy: 0.6915 - val_loss: 0.5755 - val_binary_crossentropy: 0.5406 - val_accuracy: 0.6970
Epoch 894/9000
20/20 - 1s - loss: 0.5949 - binary_crossentropy: 0.5600 - accuracy: 0.6897 - val_loss: 0.5763 - val_binary_crossentropy: 0.5414 - val_accuracy: 0.7010
Epoch 895/9000
20/20 - 1s - loss: 0.5925 - binary_crossentropy: 0.5575 - accuracy: 0.6905 - val_loss: 0.5763 - val_binary_crossentropy: 0.5414 - val_accuracy: 0.7040
Epoch 896/9000
20/20 - 1s - loss: 0.5926 - binary_crossentropy: 0.5576 - accuracy: 0.6923 - val_loss: 0.5816 - val_binary_crossentropy: 0.5466 - val_accuracy: 0.7080
Epoch 897/9000
20/20 - 1s - loss: 0.5955 - binary_crossentropy: 0.5605 - accuracy: 0.6891 - val_loss: 0.5762 - val_binary_crossentropy: 0.5412 - val_accuracy: 0.7060
Epoc

Epoch 942/9000
20/20 - 1s - loss: 0.5899 - binary_crossentropy: 0.5543 - accuracy: 0.6948 - val_loss: 0.5802 - val_binary_crossentropy: 0.5446 - val_accuracy: 0.6920
Epoch 943/9000
20/20 - 0s - loss: 0.5919 - binary_crossentropy: 0.5564 - accuracy: 0.6893 - val_loss: 0.5781 - val_binary_crossentropy: 0.5425 - val_accuracy: 0.6920
Epoch 944/9000
20/20 - 0s - loss: 0.5918 - binary_crossentropy: 0.5562 - accuracy: 0.6910 - val_loss: 0.5744 - val_binary_crossentropy: 0.5389 - val_accuracy: 0.6980
Epoch 945/9000
20/20 - 1s - loss: 0.5895 - binary_crossentropy: 0.5539 - accuracy: 0.6962 - val_loss: 0.5739 - val_binary_crossentropy: 0.5384 - val_accuracy: 0.6990
Epoch 946/9000
20/20 - 1s - loss: 0.5940 - binary_crossentropy: 0.5584 - accuracy: 0.6845 - val_loss: 0.5763 - val_binary_crossentropy: 0.5407 - val_accuracy: 0.7030
Epoch 947/9000
20/20 - 1s - loss: 0.5926 - binary_crossentropy: 0.5570 - accuracy: 0.6957 - val_loss: 0.5760 - val_binary_crossentropy: 0.5403 - val_accuracy: 0.6970
Epoc

Epoch 992/9000
20/20 - 1s - loss: 0.5911 - binary_crossentropy: 0.5550 - accuracy: 0.6965 - val_loss: 0.5832 - val_binary_crossentropy: 0.5471 - val_accuracy: 0.6880
Epoch 993/9000
20/20 - 0s - loss: 0.5932 - binary_crossentropy: 0.5572 - accuracy: 0.6892 - val_loss: 0.5806 - val_binary_crossentropy: 0.5446 - val_accuracy: 0.6930
Epoch 994/9000
20/20 - 0s - loss: 0.5911 - binary_crossentropy: 0.5551 - accuracy: 0.6970 - val_loss: 0.5741 - val_binary_crossentropy: 0.5382 - val_accuracy: 0.7010
Epoch 995/9000
20/20 - 1s - loss: 0.5925 - binary_crossentropy: 0.5566 - accuracy: 0.6975 - val_loss: 0.5720 - val_binary_crossentropy: 0.5360 - val_accuracy: 0.6980
Epoch 996/9000
20/20 - 1s - loss: 0.5895 - binary_crossentropy: 0.5535 - accuracy: 0.6937 - val_loss: 0.5761 - val_binary_crossentropy: 0.5402 - val_accuracy: 0.7030
Epoch 997/9000
20/20 - 1s - loss: 0.5902 - binary_crossentropy: 0.5542 - accuracy: 0.6932 - val_loss: 0.5795 - val_binary_crossentropy: 0.5436 - val_accuracy: 0.7000
Epoc

Epoch 1042/9000
20/20 - 0s - loss: 0.5895 - binary_crossentropy: 0.5528 - accuracy: 0.6950 - val_loss: 0.5794 - val_binary_crossentropy: 0.5428 - val_accuracy: 0.7110
Epoch 1043/9000
20/20 - 0s - loss: 0.5877 - binary_crossentropy: 0.5511 - accuracy: 0.6964 - val_loss: 0.5807 - val_binary_crossentropy: 0.5441 - val_accuracy: 0.7040
Epoch 1044/9000
20/20 - 1s - loss: 0.5881 - binary_crossentropy: 0.5515 - accuracy: 0.6941 - val_loss: 0.5818 - val_binary_crossentropy: 0.5451 - val_accuracy: 0.6940
Epoch 1045/9000
20/20 - 1s - loss: 0.5925 - binary_crossentropy: 0.5559 - accuracy: 0.6962 - val_loss: 0.5804 - val_binary_crossentropy: 0.5438 - val_accuracy: 0.6970
Epoch 1046/9000
20/20 - 1s - loss: 0.5908 - binary_crossentropy: 0.5543 - accuracy: 0.6918 - val_loss: 0.5798 - val_binary_crossentropy: 0.5432 - val_accuracy: 0.6930
Epoch 1047/9000
20/20 - 1s - loss: 0.5898 - binary_crossentropy: 0.5533 - accuracy: 0.6954 - val_loss: 0.5813 - val_binary_crossentropy: 0.5447 - val_accuracy: 0.695

20/20 - 1s - loss: 0.5860 - binary_crossentropy: 0.5489 - accuracy: 0.6939 - val_loss: 0.5818 - val_binary_crossentropy: 0.5446 - val_accuracy: 0.6900
Epoch 1092/9000
20/20 - 0s - loss: 0.5903 - binary_crossentropy: 0.5532 - accuracy: 0.6929 - val_loss: 0.5801 - val_binary_crossentropy: 0.5430 - val_accuracy: 0.6970
Epoch 1093/9000
20/20 - 1s - loss: 0.5933 - binary_crossentropy: 0.5562 - accuracy: 0.6970 - val_loss: 0.5918 - val_binary_crossentropy: 0.5547 - val_accuracy: 0.6870
Epoch 1094/9000
20/20 - 1s - loss: 0.5923 - binary_crossentropy: 0.5552 - accuracy: 0.6928 - val_loss: 0.5902 - val_binary_crossentropy: 0.5531 - val_accuracy: 0.6920
Epoch 1095/9000
20/20 - 0s - loss: 0.5864 - binary_crossentropy: 0.5493 - accuracy: 0.6964 - val_loss: 0.5798 - val_binary_crossentropy: 0.5428 - val_accuracy: 0.6950
Epoch 1096/9000
20/20 - 0s - loss: 0.5915 - binary_crossentropy: 0.5545 - accuracy: 0.6948 - val_loss: 0.5788 - val_binary_crossentropy: 0.5417 - val_accuracy: 0.7080
Epoch 1097/900

    and check its performances on the test set

In [8]:
test_accuracy = tf.keras.metrics.Accuracy()

for (features, labels) in test_ds:
    logits = regularization_model(features)
    probabilities = tf.keras.activations.sigmoid(logits)
    predictions = 1*(probabilities.numpy() > 0.5)
    test_accuracy(predictions, labels)
    regularization_model_accuracy = test_accuracy.result()

print("Test set accuracy: {:.3%}".format(regularization_model_accuracy))

Test set accuracy: 68.600%


    d) Visualize variables with tensorboard

In [11]:
%tensorboard --logdir logs/fit